<a href="https://colab.research.google.com/github/Gaurav1401/Deep-Learning/blob/main/Practice/CNN/TMLC/102_Category_Flower_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Downloading and Unzipping the Files**

In [1]:
!wget "https://s3.amazonaws.com/fast-ai-imageclas/oxford-102-flowers.tgz"

--2021-10-28 15:49:54--  https://s3.amazonaws.com/fast-ai-imageclas/oxford-102-flowers.tgz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.170.120
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.170.120|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 345236087 (329M) [application/x-tar]
Saving to: ‘oxford-102-flowers.tgz’

oxford-102-flowers. 100%[===================>] 329.24M  61.6MB/s    in 5.8s    

2021-10-28 15:50:00 (56.4 MB/s) - ‘oxford-102-flowers.tgz’ saved [345236087/345236087]



In [48]:
# Unzipping the file
# !tar -xzvf "/content/oxford-102-flowers.tgz"

# **Checking the folder content**

In [3]:
!ls oxford-102-flowers # list of files in the folder

jpg  test.txt  train.txt  valid.txt


In [4]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pathlib # to access files inside folders eaasily
import os
from tqdm import tqdm # for graph like progress bars

In [5]:
len(os.listdir("/content/oxford-102-flowers/jpg")) # checking number of image files in the jpg folder

8189

In [6]:
path = pathlib.Path("oxford-102-flowers")
all_paths = path.glob("*/*.jpg")
all_paths

<generator object Path.glob at 0x7f784ef07b50>

In [7]:
all_paths = list(all_paths)
all_paths[:5]

[PosixPath('oxford-102-flowers/jpg/image_07467.jpg'),
 PosixPath('oxford-102-flowers/jpg/image_04143.jpg'),
 PosixPath('oxford-102-flowers/jpg/image_04369.jpg'),
 PosixPath('oxford-102-flowers/jpg/image_00942.jpg'),
 PosixPath('oxford-102-flowers/jpg/image_06737.jpg')]

In [8]:
all_paths = list(map(lambda x : str(x), all_paths))
all_paths[:5]

['oxford-102-flowers/jpg/image_07467.jpg',
 'oxford-102-flowers/jpg/image_04143.jpg',
 'oxford-102-flowers/jpg/image_04369.jpg',
 'oxford-102-flowers/jpg/image_00942.jpg',
 'oxford-102-flowers/jpg/image_06737.jpg']

**Reading the text files**

In [9]:
def Read_Two_Column_File(file_name):
    with open(file_name, 'r') as data:
        x = []
        y = []
        for line in data:
            p = line.split()
            x.append(p[0])
            y.append(int(p[1]))

    return x, y

In [10]:
X_train, y_train = Read_Two_Column_File('/content/oxford-102-flowers/train.txt')
X_valid, y_valid = Read_Two_Column_File("/content/oxford-102-flowers/valid.txt")

In [11]:
X_train[:5]

['jpg/image_03860.jpg',
 'jpg/image_06092.jpg',
 'jpg/image_02400.jpg',
 'jpg/image_02852.jpg',
 'jpg/image_07710.jpg']

In [12]:
def complete_path(image_path):
  return "oxford-102-flowers/" + image_path

**Making the image path complete in the list**

In [13]:
X_train = list(map(complete_path, X_train))
X_valid = list(map(complete_path, X_valid))

In [14]:
X_train[:5]

['oxford-102-flowers/jpg/image_03860.jpg',
 'oxford-102-flowers/jpg/image_06092.jpg',
 'oxford-102-flowers/jpg/image_02400.jpg',
 'oxford-102-flowers/jpg/image_02852.jpg',
 'oxford-102-flowers/jpg/image_07710.jpg']

In [15]:
y_train[:5]

[16, 13, 42, 55, 96]

### **One hot encoding the target variables**

In [16]:
train_image_labels = tf.keras.utils.to_categorical(y_train)

train_image_labels[:10]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [17]:
valid_image_labels = tf.keras.utils.to_categorical(y_valid)

valid_image_labels[:10]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [18]:
# Function used for Transformation

def load(image , label):
    image = tf.io.read_file(image)
    image = tf.io.decode_jpeg(image , channels = 3)
    return image , label

In [19]:
# Define IMAGE SIZE and BATCH SIZE 
IMG_SIZE = 224 
BATCH_SIZE = 32

# Basic Transformation
resize = tf.keras.Sequential([
    tf.keras.layers.experimental.preprocessing.Resizing(IMG_SIZE, IMG_SIZE)          
])

# Data Augmentation
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.experimental.preprocessing.RandomFlip("horizontal"),
    tf.keras.layers.experimental.preprocessing.RandomRotation(0.2),
    tf.keras.layers.experimental.preprocessing.RandomZoom(height_factor = (-0.3, -0.2))
])

In [20]:
# Function used to Create a Tensorflow Data Object
AUTOTUNE = tf.data.experimental.AUTOTUNE
def get_dataset(paths , labels , train = True):
    image_paths = tf.convert_to_tensor(paths)
    labels = tf.convert_to_tensor(labels)

    image_dataset = tf.data.Dataset.from_tensor_slices(image_paths)
    label_dataset = tf.data.Dataset.from_tensor_slices(labels)

    dataset = tf.data.Dataset.zip((image_dataset , label_dataset))

    dataset = dataset.map(lambda image , label : load(image , label))
    dataset = dataset.map(lambda image, label: (resize(image), label) , num_parallel_calls=AUTOTUNE)
    dataset = dataset.shuffle(1000)
    dataset = dataset.batch(BATCH_SIZE)

    if train:
        dataset = dataset.map(lambda image, label: (data_augmentation(image), label) , num_parallel_calls=AUTOTUNE)
    
    dataset = dataset.repeat()
    return dataset

In [21]:
# Creating Train Dataset object and Verifying it
%time train_dataset = get_dataset(X_train , train_image_labels)

image , label = next(iter(train_dataset))
print(image.shape)
print(label.shape)

CPU times: user 524 ms, sys: 14 ms, total: 538 ms
Wall time: 566 ms
(32, 224, 224, 3)
(32, 102)


In [22]:
%time val_dataset = get_dataset(X_valid , valid_image_labels , train = False)

image , label = next(iter(val_dataset))
print(image.shape)
print(label.shape)

CPU times: user 22.9 ms, sys: 3.69 ms, total: 26.6 ms
Wall time: 29 ms
(32, 224, 224, 3)
(32, 102)


In [36]:
# Model part 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Conv2D, MaxPooling2D, Flatten, Dropout, BatchNormalization , GlobalMaxPool2D
from tensorflow.keras.callbacks import EarlyStopping


model = Sequential()

# Block 1 
model.add(Conv2D(input_shape=(224 , 224 , 3),  padding='same',filters=32, kernel_size=(7, 7)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

# Block 2
model.add(Conv2D(filters=64,  padding='valid', kernel_size=(5, 5)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

# Block 3 
model.add(Conv2D(filters=128, padding='valid', kernel_size=(3, 3)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D())
model.add(Dropout(0.2))

# Block 4 
model.add(Conv2D(filters=256, padding='valid', kernel_size=(3, 3)))
model.add(Activation('relu'))
model.add(BatchNormalization())

model.add(Conv2D(filters=256 , kernel_size=(3, 3)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

model.add(GlobalMaxPool2D())

model.add(Dense(units=256))
model.add(Activation('relu'))
model.add(Dropout(0.2))

model.add(Dense(units=102))
model.add(Activation('softmax'))

In [37]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_10 (Conv2D)           (None, 224, 224, 32)      4736      
_________________________________________________________________
activation_14 (Activation)   (None, 224, 224, 32)      0         
_________________________________________________________________
batch_normalization_10 (Batc (None, 224, 224, 32)      128       
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 112, 112, 32)      0         
_________________________________________________________________
dropout_10 (Dropout)         (None, 112, 112, 32)      0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 108, 108, 64)      51264     
_________________________________________________________________
activation_15 (Activation)   (None, 108, 108, 64)     

In [40]:
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [41]:
early_stop = EarlyStopping(monitor='val_loss', patience=2)

In [42]:
history = model.fit(
    train_dataset,
    steps_per_epoch=len(X_train)//BATCH_SIZE,
    epochs=20,
    validation_data=val_dataset,
    validation_steps = len(X_valid)//BATCH_SIZE,
    callbacks = early_stop
)

Epoch 1/20
31/31 [==============================] - 19s 476ms/step - loss: 6.1688 - accuracy: 0.0192 - val_loss: 21.7370 - val_accuracy: 0.0171
Epoch 2/20
31/31 [==============================] - 16s 544ms/step - loss: 4.3125 - accuracy: 0.0526 - val_loss: 6.3691 - val_accuracy: 0.0262
Epoch 3/20
31/31 [==============================] - 17s 550ms/step - loss: 4.0276 - accuracy: 0.0749 - val_loss: 4.2988 - val_accuracy: 0.0423
Epoch 4/20
31/31 [==============================] - 16s 535ms/step - loss: 3.8517 - accuracy: 0.0840 - val_loss: 4.3089 - val_accuracy: 0.0575
Epoch 5/20
31/31 [==============================] - 16s 538ms/step - loss: 3.6796 - accuracy: 0.1093 - val_loss: 3.7111 - val_accuracy: 0.1079
Epoch 6/20
31/31 [==============================] - 16s 542ms/step - loss: 3.5681 - accuracy: 0.1437 - val_loss: 3.7456 - val_accuracy: 0.1179
Epoch 7/20
31/31 [==============================] - 16s 540ms/step - loss: 3.3857 - accuracy: 0.1721 - val_loss: 3.7693 - val_accuracy: 0.109

In [47]:
# Model Evaluation
loss, acc = model.evaluate(val_dataset, steps=50)

print(" Testing Acc : " , acc)
print(" Testing Loss : " , loss)

50/50 [==============================] - 7s 104ms/step - loss: 3.7725 - accuracy: 0.1109
 Testing Acc :  0.11090225726366043
 Testing Loss :  3.772536277770996
